In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

c:\Users\alber\Desktop\projetos\Generative-IA\venv-genai2\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alber\.cache\huggingface\hub\models--Babelscape--wikineural-multilingual-ner. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu


[{'entity_group': 'PER', 'score': np.float32(0.96688986), 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity_group': 'LOC', 'score': np.float32(0.9996282), 'word': 'Berlin', 'start': 34, 'end': 40}]


c:\Users\alber\Desktop\projetos\Generative-IA\venv-genai2\Lib\site-packages\transformers\pipelines\token_classification.py:181: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)

In [31]:
example = "Meu nome é João e moro em São Paulo. Meu email é joao.souza@gmail.com"

ner_results = nlp(example)
print(ner_results)

[{'entity_group': 'PER', 'score': np.float32(0.8182017), 'word': 'João', 'start': 11, 'end': 15}, {'entity_group': 'LOC', 'score': np.float32(0.9997442), 'word': 'São Paulo', 'start': 26, 'end': 35}]


In [ ]:
from transformers import (
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)

model = AutoModelForTokenClassification.from_pretrained(
    model_name, num_labels=num_entity_labels
)
args = TrainingArguments(
    output_dir="./ner_finetuned",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    num_train_epochs=3,
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=200,
    save_steps=200
)
data_collator = DataCollatorForTokenClassification(tokenizer)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)
trainer.train()


In [29]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline

In [12]:
questions = [
    "Qual foi a quantidade vendida de Leite Integral?",
    "Qual o valor total de vendas de Arroz Tipo 1?",
    "Quais produtos vencerão antes de 2025-12-01?",
    "Qual fornecedor tem o maior total de vendas reais?",
    "Quantos Pães de Forma foram vendidos até agora?",
    "Liste os produtos do tipo 'Óleo' e suas quantidades vendidas.",
    "Qual o preço unitário do Feijão Preto?",
    "Quais produtos têm estoque vencido (data de vencimento anterior a hoje)?",
    "Qual a soma das vendas reais para produtos do fornecedor 'Padaria Central'?",
    "Quais produtos têm quantidade vendida maior que 100 unidades?"
]

# 2) SQL correspondentes
sql_queries = [
    "SELECT quantidade_vendida FROM produtos WHERE nome_produto = 'Leite Integral';",
    "SELECT SUM(vendas_reais) FROM produtos WHERE nome_produto = 'Arroz Tipo 1';",
    "SELECT nome_produto, data_vencimento FROM produtos WHERE data_vencimento < '2025-12-01';",
    "SELECT fornecedor, SUM(vendas_reais) as total_vendas FROM produtos GROUP BY fornecedor ORDER BY total_vendas DESC LIMIT 1;",
    "SELECT quantidade_vendida FROM produtos WHERE nome_produto = 'Pão de Forma';",
    "SELECT nome_produto, quantidade_vendida FROM produtos WHERE tipo = 'Óleo';",
    "SELECT preco_unitario FROM produtos WHERE nome_produto = 'Feijão Preto';",
    "SELECT nome_produto, data_vencimento FROM produtos WHERE data_vencimento < CURRENT_DATE;",
    "SELECT SUM(vendas_reais) FROM produtos WHERE fornecedor = 'Padaria Central';",
    "SELECT nome_produto, quantidade_vendida FROM produtos WHERE quantidade_vendida > 100;"
]

# 3) Mapeamento Pergunta -> SQL
sql_mapping = dict(zip(questions, sql_queries))

In [ ]:
# 3) Carregar o modelo multilíngue
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

c:\Users\alber\Desktop\projetos\Generative-IA\venv-genai2\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alber\Desktop\projetos\Generative-IA\models\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' pa

In [4]:
# 4) Gerar embeddings em float32
embs = model.encode(questions, convert_to_numpy=True, show_progress_bar=False).astype('float32')

In [5]:
# 5) Construir índice ANN (IVF Flat)
d = embs.shape[1]          # dimensão dos embeddings
n_clusters = 5             # ajuste conforme volume de dados
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, n_clusters, faiss.METRIC_L2)

index.train(embs)          # treina o k-means
index.add(embs)            # adiciona os embeddings
index.nprobe = 2           # quantos clusters vasculhar

In [18]:
# 6) Exemplo de busca
query = "Quanto vendemos de Leite Integral até agora?"
q_emb = model.encode([query], convert_to_numpy=True).astype('float32')
# Busca aproximada dos 3 vizinhos mais próximos
D, I = index.search(q_emb, k=3)

# 7) Exibir resultados
print("Query:", query)
print("\nTop 3 resultados (índice, pergunta, distância L2):")
for rank, idx in enumerate(I[0], start=1):
    print(f"{rank}. (idx {idx}) {questions[idx]} — distância {D[0][rank-1]:.4f}")

Query: Quanto vendemos de Leite Integral até agora?

Top 3 resultados (índice, pergunta, distância L2):
1. (idx 0) Qual foi a quantidade vendida de Leite Integral? — distância 0.2380
2. (idx 4) Quantos Pães de Forma foram vendidos até agora? — distância 0.9728
3. (idx 6) Qual o preço unitário do Feijão Preto? — distância 1.2577


In [10]:
# 1) Reextraia os embeddings float32 dos candidatos
candidate_embs = embs[I[0]]  # shape (k, D)

# 2) Calcule cosine similarity
scores = cosine_similarity(q_emb, candidate_embs)[0]  # shape (k,)

# 3) Reordene os candidatos por score
order = np.argsort(-scores)
top_k_ordered = [(I[0][i], scores[i]) for i in order]

print("Reranking (idx, score):")
for idx, score in top_k_ordered:
    print(f"  idx={idx}, pergunta='{questions[idx]}' → score={score:.4f}")

Reranking (idx, score):
  idx=0, pergunta='Qual foi a quantidade vendida de Leite Integral?' → score=0.8810
  idx=4, pergunta='Quantos Pães de Forma foram vendidos até agora?' → score=0.5136
  idx=6, pergunta='Qual o preço unitário do Feijão Preto?' → score=0.3712


In [13]:
tau = 0.85
best_idx, best_score = top_k_ordered[0]

if best_score >= tau:
    # Hit no cache
    cached_question = questions[best_idx]
    cached_sql      = sql_mapping[cached_question]
    print("Usando cache:", cached_sql)
else:
    # Sem hit confiável
    print("Nenhum candidato passa no limiar; seguir para o LLM.")


Usando cache: SELECT quantidade_vendida FROM produtos WHERE nome_produto = 'Leite Integral';


In [22]:
ner = pipeline("ner", model="Davlan/bert-base-multilingual-cased-ner-hrl",
               aggregation_strategy="simple")

Device set to use cpu


In [27]:
def parse_question(text):
    # 1) Identifica intent
    #intent = intent_classifier(text)[0]['label']
    
    # 2) Roda NER e monta dicionário de slots
    entities = ner(text)
    slots = {}
    for ent in entities:
        label = ent['entity_group']  # ex: 'ORG', 'DATE', 'MISC'
        # mapeie labels para slots do seu template, por ex:
        if label in ('ORG','MISC'):
            slots['PRODUTO'] = ent['word']
        elif label == 'DATE':
            slots['DATA'] = ent['word']
        # etc.
    return slots


In [28]:
parse_question(query)

{}